# Exercises for 2021-02-16

In these exercises, we will look at regularization for least squares in the context of a curve fitting problem.  Before we begin, it may be helpful to know a little about the [Legendre polynomails](https://en.wikipedia.org/wiki/Legendre_polynomials), which play an important role in approximation
theory and in the development of Gaussian quadrature.  Legendre polynomials may be computed by the recurrence
$$
  (n+1) P_{n+1}(x) = (2n+1)x P_n(x) - n P_{n-1}(x)
$$
where $P_0(x) = 1$ and $P_1(x) = x$.  These polynomials are orthogonal with respect to the $L^2$ inner product
on $[-1, 1]$:
$$
  \langle P_m, P_n \rangle_{L_2} = \int_{-1}^1 P_m(x) P_n(x) \, dx = \frac{2}{2n+1} \delta_{mn}.
$$
The normalized Legendre polynomials are
$$
  Q_n(x) = \sqrt{\frac{2n+1}{2}} P_n(x),
$$
and they form an orthonormal basis for $L^2$ on $[-1,1]$.

In [ ]:
using LinearAlgebra
using Plots

In [ ]:
function legendre(xx, dmax)
    P = zeros(length(xx), dmax+1)
    P[:,1] .= 1.0
    if dmax > 0
        P[:,2] .= xx
    end
    for n = 1:dmax-1
        P[:,n+2] .= ( (2*n+1) .* xx .* P[:,n+1] - n * P[:,n] )/(n+1)
    end
    return P
end

function nlegendre(xx, dmax)
    Q = legendre(xx, dmax)
    for n = 0:dmax
        Q[:,n+1] .*= sqrt(n+0.5)
    end
    return Q
end

In [ ]:
xx = range(-1, 1, length=100)
plot(xx, nlegendre(xx, 5))

*TODO*: Suppose $g(x) = \sum_{j=0}^d c_j Q_j(x)$, where the $Q_j$ are the normalized Legendre polynomials described above.  Then $\|g\|^2_{L^2} \equiv \int_{-1}^1 g(x)^2 \, dx$ and $\|c\|^2 = \sum_{j=0}^d c_j^2$ are the same.  Why?

Now consider the problem of approximating a simple bump function.

In [ ]:
f(x) = exp(-20.0*x^2)
plot(xx, f.(xx))

It turns out that approximating this function by interpolation on an equi-spaced grid gives bad results near the end of the interval due to [Runge's phenomenon](https://en.wikipedia.org/wiki/Runge%27s_phenomenon).

In [ ]:
dmax = 20
xs = range(-1.0, 1.0, length=dmax+1)
A = nlegendre(xs, dmax)
b = f.(xs)
c = A\b
print("Max error: $(norm(f.(xx).-nlegendre(xx, dmax)*c, Inf))")
plot(xx, f.(xx))
plot!(xx, nlegendre(xx, dmax)*c)

This turns out to be an *ill-conditioned* problem, but increasing the number of data points helps a great deal.

In [ ]:
ns = range(3, 60, step=1)
nlegendreA(n, dmax) = nlegendre(range(-1.0, 1.0, length=n), dmax)
plot(ns, cond.(nlegendreA.(ns, dmax)), yscale=:log10, xlabel="Number of points", ylabel="Conditioning of LS")

In [ ]:
dmax = 20
xs = range(-1.0, 1.0, length=201)
A = nlegendre(xs, dmax)
b = f.(xs)
c_hi = A\b
print("Max error: $(norm(f.(xx).-nlegendre(xx, dmax)*c_hi, Inf))")
plot(xx, f.(xx))
plot!(xx, nlegendre(xx, dmax)*c_hi)

## Regularization

Now, suppose that we have data at 21 points for a degree 20 polynomial, and we aren't able to get more data.  Let's consider two different forms of regularization: adaptively choosing a polynomial degree, or Tikhonov regularization.  In each case, we use a [cross-validation statistic (LOOCV)](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) (also known as the [PRESS statistic](https://en.wikipedia.org/wiki/PRESS_statistic)) to judge our fit.

### Choosing the polynomial degree

If we were able to evaluate the error on a dense grid, a natural way to choose the polynomial degree for fitting to 21 points would be to choose it to minimize the error.  In this problem, since we have a ground truth, we can do this; in general, though, we don't like methods that require us to use much more data for evaluation than for fitting!

In [ ]:
dmax = 20
npts = dmax+1
xs = range(-1.0, 1.0, length=npts)
#xs = 2.0*rand(dmax+1).-1.0
A = nlegendre(xs, dmax)
b = f.(xs)
Q, R = qr(A)
QTb = Q'*b

Ahi = nlegendre(xx, dmax)
err(k) = norm(f.(xx) - Ahi[:,1:k]*(R[1:k,1:k]\QTb[1:k]), Inf)
ks = range(1, dmax, step=1)
scatter(ks, err.(ks), yscale=:log10, xlabel="Degree", ylabel="Max error on mesh")

The idea of leave-one-out cross validation (LOOCV) is that we check each data point on a model trained without
that data point.  That is
$$
  \mbox{LOOCV} = \frac{1}{n} \sum_{i=1}^n (b_i - b_{[-i]})^2
$$
where $b_{[-i]}$ is the prediction at point $i$ based on all the data except that at point $i$.

We start by doing this the slow way (which is not so slow for this small a problem, admittedly).

In [ ]:
slow_loocv = zeros(dmax)
for k = 1:dmax
    for i = 1:npts
        mask = BitArray(i != j for j = 1:npts)
        cik = A[mask, 1:k]\b[mask]
        slow_loocv[k] += (b[i] - dot(A[i,1:k], cik))^2/npts
    end
end
scatter(ks, slow_loocv, yscale=:log10)

For least squares, there is a clever linear algebra trick that allows us to compute the LOOCV statistic without actually computing the leave-one-out fit for each of the data points from scratch.  The key is to compute the diagonal of the "influence matrix" $H$ that maps the data points to the vector of model evaluations at the data points (these values are sometimes called the [leverage scores](https://en.wikipedia.org/wiki/Leverage_(statistics)) of the data points, and play an important role in outlier detection as well).  In terms of this matrix, we have
$$
  \mbox{LOOCV} = \frac{1}{n} \sum_{j=1}^n \left( \frac{r_j}{1-h_{jj}} \right)^2
$$
where $r = (I-H)b$ is the residual vector and $h_{jj}$ is the $j$th diagonal entry of $H$.

If we use the economy QR factorization $A=QR$ for a rectangular $A$, the influence matrix is $H = QQ^T$, the least squares projection onto the range space of $A$.  Thus, the economy QR factorization gives us not only a fast way to compute the least squares solution, but also a fast way to compute the LOOCV statistic.

*TODO* (2): Fill in the following code to compute the LOOCV using the fast algorithm.  You should be able to do this by updating the residual vector $r$ and diagonal vector $h$ in $O(n)$ time per step of the main loop (forming $Q$ expicitly is the most expensive piece of this code).

In [ ]:
Q = Array(Q)
loocv = zeros(dmax)
h = zeros(npts)
r = b
for k = 1:dmax
    # TODO: Update r
    # TODO: Update h
    loocv[k] = sum( (r./(1.0.-h)).^2 )/npts
end

print("Max relerr: $(norm((slow_loocv-loocv)./slow_loocv, Inf))")

### Choosing a Tikhonov parameter

In the case of Tikhonov regularization, we again start by cheating and taking advantage of the fact that we know the "ground truth."  For small problems, the fastest way to do this is via the SVD.

*TODO*: Explain the `tikhonov_fit` function below.

In [ ]:
A = nlegendre(xs, dmax)
UA, SA, VA = svd(A)
tikhonov_fit(b, λ) = VA*((UA'*b).*(SA./(SA.^2 .+ λ^2)))

Ahi = nlegendre(xx, dmax)

err_tik(λ) = norm(f.(xx) - Ahi*tikhonov_fit(b, λ), Inf)
λs = exp10.(range(-6, 3, step=0.2))
scatter(λs, err_tik.(λs), xscale=:log10, yscale=:log10, xlabel="Lambda", ylabel="Max error on mesh")

*TODO*: Explain the code below, which computes the LOOCV statistic for the regularized least squares problem.

In [ ]:
loocv = zeros(length(λs))
UTb = UA'*b
h = zeros(npts)
for k = 1:length(λs)
    λ = λs[k]
    HS = SA.^2 ./ (SA.^2 .+ λ^2)
    r = b - UA*(HS.*UTb)
    h = [sum(UA[i,:].^2 .* HS) for i in 1:npts]
    loocv[k] = sum(( r ./ (1.0.-h) ).^2)/npts
end
scatter(λs, loocv, xscale=:log10, yscale=:log10)